## Objective
This notebook focuses on cleaning and preparing the credit card customer dataset
for exploratory analysis and customer segmentation. The steps include column
screening, missing value treatment, and creation of a clean dataset for downstream analysis.



In [9]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

df = pd.read_csv("../data/raw/CreditCard_STM.csv")
df.head()

,Unnamed: 0,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,693,C10719,908.354652,1.000000,231.35,0.00,231.35,944.183414,1.000000,0.000000,0.857143,0.142857,3,7,1000.0,161.975360,930.115009,0.0,7
1,6277,C16451,744.083484,1.000000,1096.98,589.48,507.50,174.714236,1.000000,0.500000,0.600000,0.200000,4,14,5000.0,1873.388189,148.294074,0.1,10
2,7577,C17782,738.067431,1.000000,752.97,752.97,0.00,0.000000,0.666667,0.666667,0.000000,0.000000,0,11,2500.0,1393.051729,282.663936,0.0,12
3,5981,C16146,333.085021,0.636364,980.00,980.00,0.00,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1000.0,1485.205378,163.396716,0.0,12
4,6748,C16935,3559.634883,1.000000,7654.81,6984.29,670.52,0.000000,0.666667,0.666667,0.333333,0.000000,0,45,7000.0,2785.055072,783.737906,0.0,12


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        8800 non-null   int64  
 1   CUST_ID                           8800 non-null   object 
 2   BALANCE                           8800 non-null   float64
 3   BALANCE_FREQUENCY                 8800 non-null   float64
 4   PURCHASES                         8800 non-null   float64
 5   ONEOFF_PURCHASES                  8800 non-null   float64
 6   INSTALLMENTS_PURCHASES            8800 non-null   float64
 7   CASH_ADVANCE                      8800 non-null   float64
 8   PURCHASES_FREQUENCY               8800 non-null   float64
 9   ONEOFF_PURCHASES_FREQUENCY        8800 non-null   float64
 10  PURCHASES_INSTALLMENTS_FREQUENCY  8800 non-null   float64
 11  CASH_ADVANCE_FREQUENCY            8800 non-null   float64
 12  CASH_A

### Column Overview

The dataset contains customer-level aggregated behavioral features, which can be grouped as:

- **Identifiers**
  - `CUST_ID`: Unique customer identifier

- **Balance & Spending Amounts**
  - `BALANCE`, `PURCHASES`, `ONEOFF_PURCHASES`, `INSTALLMENTS_PURCHASES`
  - `CASH_ADVANCE`, `PAYMENTS`, `MINIMUM_PAYMENTS`, `CREDIT_LIMIT`

- **Behavioral Frequencies**
  - `BALANCE_FREQUENCY`, `PURCHASES_FREQUENCY`
  - `ONEOFF_PURCHASES_FREQUENCY`, `PURCHASES_INSTALLMENTS_FREQUENCY`
  - `CASH_ADVANCE_FREQUENCY`

- **Transaction Counts & Ratios**
  - `PURCHASES_TRX`, `CASH_ADVANCE_TRX`
  - `PRC_FULL_PAYMENT`, `TENURE`


### Column Removal

The following columns are removed prior to analysis:
- `Unnamed: 0`: Index column unintentionally saved during CSV export
- `CUST_ID`: Identifier with no analytical value for segmentation

In [10]:
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df = df.drop(columns=["CUST_ID"])

In [11]:
df.isnull().mean().sort_values(ascending=False)

MINIMUM_PAYMENTS                    0.034659
CREDIT_LIMIT                        0.000114
BALANCE                             0.000000
CASH_ADVANCE_FREQUENCY              0.000000
PRC_FULL_PAYMENT                    0.000000
PAYMENTS                            0.000000
PURCHASES_TRX                       0.000000
CASH_ADVANCE_TRX                    0.000000
PURCHASES_INSTALLMENTS_FREQUENCY    0.000000
BALANCE_FREQUENCY                   0.000000
ONEOFF_PURCHASES_FREQUENCY          0.000000
PURCHASES_FREQUENCY                 0.000000
CASH_ADVANCE                        0.000000
INSTALLMENTS_PURCHASES              0.000000
ONEOFF_PURCHASES                    0.000000
PURCHASES                           0.000000
TENURE                              0.000000
dtype: float64

The dataset contains a very limited amount of missing data.
Only two variables show missing values:

- `MINIMUM_PAYMENTS` (~3.5%)
- `CREDIT_LIMIT` (~0.01%)

All other variables are complete.
Given the low proportion of missing values, imputation is preferred over
row removal in order to preserve the full customer population.

### Missing Value Treatment Strategy

Different imputation strategies are applied depending on the distribution
and business meaning of each variable:

- **Median imputation** is used for highly skewed monetary variables
  (e.g. payments, balances), as it is more robust to extreme values and
  outliers commonly observed in financial data.

- **Mean imputation** is used for bounded ratio or frequency variables
  (e.g. payment ratios), where values are naturally constrained and
  extreme outliers are less influential.

In [15]:
# Median imputation for monetary variables with missing values
df["MINIMUM_PAYMENTS"] = df["MINIMUM_PAYMENTS"].fillna(df["MINIMUM_PAYMENTS"].median())
df["CREDIT_LIMIT"] = df["CREDIT_LIMIT"].fillna(df["CREDIT_LIMIT"].median())

In [16]:
df[["MINIMUM_PAYMENTS", "CREDIT_LIMIT"]].isnull().sum()

MINIMUM_PAYMENTS    0
CREDIT_LIMIT        0
dtype: int64

In [17]:
df.to_csv("../data/processed/customer_clean.csv", index=False)